# GFS filter service

This notebooks contains an example of downloading GFS data using the filter service, that works with the original GRIB files.

In [ ]:
import datetime as dt
from pathlib import Path
import urllib.request as request
from contextlib import closing
from pathlib import Path
import shutil
import numpy as np

range1 = lambda start, end, step=1: np.arange(start, end + step, step)


def get_gfs_grib_filter(
    date: dt.datetime,
    run: int,
    vars: set,
    levels: set,
    lat: tuple[float, float] = (-90, 90),
    lon: tuple[float, float] = (-180, 180),
    future_window: int = 8,
    outdir: str = ".",
):
    """Download meteorological data from GFS HTTP server using the grib filter
    service. This allows subsetting coordinates, time steps, variables and levels

    References
    ----------
    [1] https://nomads.ncep.noaa.gov/
    [2] https://www.nco.ncep.noaa.gov/pmb/products/gfs/
    """
    base_url = "https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25.pl"
    fout = Path(outdir) / f"gfs_{date:%Y%m%d}_{run:02d}.grb2"

    # Transform lon from -180, 180 to 0, 360. The URL of the requests
    # changes whether it is the full region or a sub-region
    lon0, lon1 = lon[0] % 360, lon[1] % 360

    if lon0 == lon1 == 180 or lon0 > lon1:
        lon0 = 0
        lon1 = 360
        prefix = ""
    else:
        prefix = "subregion=&"

    dir = f"%2Fgfs.{date:%Y%m%d}%2F{run:02d}%2Fatmos"

    coords_ = (
        f"{prefix}leftlon={lon0}&rightlon={lon1}&toplat={lat[1]}&bottomlat={lat[0]}"
    )
    levels_ = "&".join([f"lev_{level}=on" for level in levels])
    vars_ = "&".join([f"var_{var}=on" for var in vars])

    # Until hour 120 (5 days), time resolution is 1 hour. From hour 120 to 384 (16 days),
    # time resolution is 3 hours, so we need to combine two ranges in the last case
    last_hour = future_window * 24
    if last_hour <= 120:
        time_range = range1(0, last_hour)
    else:
        time_range = np.concatenate((range1(0, 120), range1(123, last_hour, 3)))

    try:
        with open(fout, "wb") as f:
            for time in time_range:
                file = f"gfs.t{run:02d}z.pgrb2.0p25.f{time:03d}"
                url = f"{base_url}?file={file}&{levels_}&{vars_}&{coords_}&dir={dir}"

                # Print URL for debugging
                if time == 0:
                    print(url)

                with closing(request.urlopen(url)) as r:
                    shutil.copyfileobj(r, f)
    except Exception as err:
        fout.unlink(missing_ok=True)
        raise IOError(f"{err}")

    print(f'File "{fout.name}" downloaded')


There are 4 runs per day (00, 06, 12, 18). Information about available variables can be found [here](https://www.nco.ncep.noaa.gov/pmb/products/gfs/gfs.t00z.pgrb2.0p25.f000.shtml). Example: download wind speed forecasts (u and v components) 10 meters above ground for today (24 values, one per hour, starting at 00).

In [2]:
date = dt.date.today()
run = 0
vars = {"UGRD", "VGRD"}
levels = {"10_m_above_ground"}

get_gfs_grib_filter(date, run, vars, levels, future_window=1)


https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25.pl?file=gfs.t00z.pgrb2.0p25.f000&lev_10_m_above_ground=on&var_VGRD=on&var_UGRD=on&leftlon=0&rightlon=360&toplat=90&bottomlat=-90&dir=%2Fgfs.20221109%2F00%2Fatmos
File "gfs_20221109_00.grb2" downloaded


Depending on the variables and levels that you download, you can open the GRIB file with xarray and export it to NetCDF. See [this](https://github.com/albertotb/get-gfs/issues/9#issuecomment-886061535) comment for more information.

In [3]:
import xarray as xr

with xr.open_dataset(f"gfs_{date:%Y%m%d}_{run:02d}.grb2", engine="cfgrib") as ds:
    ds.to_netcdf(f"gfs_{date:%Y%m%d}_{run:02d}.nc")

Ignoring index file 'gfs_20221109_00.grb2.923a8.idx' older than GRIB file
